In [1]:
import os
os.chdir('/home/jupyter-nafisha/normal-abnormal-multitask/main')

In [2]:
import torch
import torch.nn as nn
from dataset import XrayDataset
from torch.utils.data import DataLoader
from transform import train_transform
from multimodel import Multimodel

In [3]:
# pwd

In [4]:
csv_path = '/home/jupyter-nafisha/normal-abnormal-multitask/test.csv'
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

In [29]:
train_dataset = XrayDataset(img_dir='', csv_path=csv_path, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)

In [30]:
for image, disease_id, label, bbox, has_bbox in train_loader:
    image, disease_id, label, bbox, has_bbox = image, disease_id, label, bbox, has_bbox
    break

In [32]:
image.shape, label.shape, bbox, has_bbox

(torch.Size([4, 3, 224, 224]),
 torch.Size([4]),
 tensor([[   0.,    0.,    0.,    0.],
         [1266.,  770., 1911., 1361.],
         [1194., 1458., 1496., 1820.],
         [ 920., 1775., 2034., 2231.]]),
 tensor([False,  True,  True,  True]))

In [33]:
disease_id, label, bbox, has_bbox

(tensor([ 0, 10, 11,  5]),
 tensor([0., 1., 1., 1.]),
 tensor([[   0.,    0.,    0.,    0.],
         [1266.,  770., 1911., 1361.],
         [1194., 1458., 1496., 1820.],
         [ 920., 1775., 2034., 2231.]]),
 tensor([False,  True,  True,  True]))

In [34]:
# import matplotlib.pyplot as plt

# # batch: torch.Tensor of shape [4, 3, 224, 224]
# batch = image.cpu()   # make sure it's on CPU

# for i in range(batch.shape[0]):
#     img = batch[i].permute(1, 2, 0)  # CHW → HWC

#     plt.figure()
#     plt.imshow(img)
#     plt.axis("off")
#     plt.title(f"Image {i}")
#     plt.show()


In [35]:
model = Multimodel().to(device)

/home/jupyter-nafisha/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jupyter-nafisha/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [36]:
image = image.to(device)
label = label.to(device)
bbox  = bbox.to(device)
has_bbox = has_bbox.to(device)

In [37]:
bbox

tensor([[   0.,    0.,    0.,    0.],
        [1266.,  770., 1911., 1361.],
        [1194., 1458., 1496., 1820.],
        [ 920., 1775., 2034., 2231.]])

In [64]:
criterian = nn.BCEWithLogitsLoss()
bbox_loss = nn.MSELoss(reduction="none")

In [55]:
# model

In [56]:
cls_logits, bbox_preds = model(image, disease_id, has_bbox, return_bbox=True)

In [57]:
cls_logits, bbox_preds

(tensor([[-0.1771],
         [ 0.0447],
         [-0.1933],
         [ 0.1004]], grad_fn=<AddmmBackward0>),
 tensor([[ 0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00],
         [ 3.7800e-01, -3.4121e-01, -8.4418e-02,  4.8902e-02],
         [ 5.5006e-02, -2.9743e-05,  2.1258e-01, -3.9568e-01],
         [ 3.6326e-01, -2.4143e-01,  3.5239e-02, -3.4893e-02]],
        grad_fn=<MulBackward0>))

In [58]:
cls_logits = model(image)
cls_logits

tensor([[-0.1027],
        [-0.0373],
        [-0.0811],
        [ 0.0921]], grad_fn=<AddmmBackward0>)

In [59]:
bbox_preds.shape, bbox.shape

(torch.Size([4, 4]), torch.Size([4, 4]))

In [65]:
loss_bbox = bbox_loss(bbox_preds, bbox)

In [66]:
loss_bbox

tensor(1869618., grad_fn=<MseLossBackward0>)

In [61]:
loss_bbox

tensor([[      0.0000,       0.0000,       0.0000,       0.0000],
        [1601798.8750,  593425.5625, 3652243.7500, 1852187.7500],
        [1425504.5000, 2125764.0000, 2237380.2500, 3313840.2500],
        [ 845731.6875, 3151482.2500, 4137012.5000, 4977517.0000]],
       grad_fn=<MseLossBackward0>)

In [71]:
mask = has_bbox.unsqueeze(1).float()
loss_bbox = (loss_bbox * mask).sum() / mask.sum()

In [72]:
loss_bbox

tensor(1869618., grad_fn=<DivBackward0>)

In [46]:
loss_bbox

tensor(1869594.3750, grad_fn=<DivBackward0>)

In [20]:
cls_logits.shape, label.shape

(torch.Size([4, 1]), torch.Size([4]))

In [22]:
label

tensor([0., 0., 0., 1.])

In [26]:
preds = (torch.sigmoid(cls_logits).squeeze(1) > 0.5).float()
preds

tensor([0., 1., 1., 1.])

In [27]:
total_correct = (preds == label).sum().item()

In [28]:
preds == label

tensor([ True, False, False,  True])

In [10]:
import shutil
dir_path = '/home/jupyter-nafisha/chexAgent-Normal-Abnormal/test_orig_CLAHE'
shutil.make_archive(dir_path, 'zip', dir_path)

'/home/jupyter-nafisha/chexAgent-Normal-Abnormal/test_orig_CLAHE.zip'

In [8]:
pwd

'/home/jupyter-nafisha/normal-abnormal-multimodel'

In [9]:
DISEASES = [
    'no_bbox',
    'No finding',
    'Aortic enlargement',
    'Atelectasis',
    'Calcification',
    'Cardiomegaly',
    'Consolidation',
    'ILD',
    'Infiltration',
    'Lung Opacity',
    'Nodule/Mass',
    'Other lesion',
    'Pleural effusion',
    'Pleural thickening',
    'Pneumothorax',
    'Pulmonary fibrosis'
]

In [10]:
disease2id = {disease: idx for idx, disease in enumerate(DISEASES)}
id2disease = {idx: disease for disease, idx in disease2id.items()}

In [11]:
disease2id

{'no_bbox': 0,
 'No finding': 1,
 'Aortic enlargement': 2,
 'Atelectasis': 3,
 'Calcification': 4,
 'Cardiomegaly': 5,
 'Consolidation': 6,
 'ILD': 7,
 'Infiltration': 8,
 'Lung Opacity': 9,
 'Nodule/Mass': 10,
 'Other lesion': 11,
 'Pleural effusion': 12,
 'Pleural thickening': 13,
 'Pneumothorax': 14,
 'Pulmonary fibrosis': 15}

In [26]:
disease= 'Other lesion'
disease2id[disease]

10

In [27]:
def encode_disease(disease):
    return disease2id[disease]

In [30]:
for disease in DISEASES:
    print(disease, encode_disease(disease))

No finding 0
Aortic enlargement 1
Atelectasis 2
Calcification 3
Cardiomegaly 4
Consolidation 5
ILD 6
Infiltration 7
Lung Opacity 8
Nodule/Mass 9
Other lesion 10
Pleural effusion 11
Pleural thickening 12
Pneumothorax 13
Pulmonary fibrosis 14
